I'm going to predict the actual temperature & what's our human body feels like in that actual temperature using PyTorch. For that, I have several features called Humidity, Pressures, Wind Speed, Wind Bearing.

In [1]:
import pandas as pd
import numpy as np
import torch
data = pd.read_csv('../input/szeged-weather/weatherHistory.csv')

# **Without Build-In Functions**

In [2]:
display(data)

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


In [3]:
input = np.array([data['Humidity'], data['Pressure (millibars)'],
                 data['Wind Speed (km/h)'], data['Wind Bearing (degrees)']],
                 dtype="float32").T
print(input)

[[8.90000e-01 1.01513e+03 1.41197e+01 2.51000e+02]
 [8.60000e-01 1.01563e+03 1.42646e+01 2.59000e+02]
 [8.90000e-01 1.01594e+03 3.92840e+00 2.04000e+02]
 ...
 [5.60000e-01 1.01566e+03 8.98380e+00 3.00000e+01]
 [6.00000e-01 1.01595e+03 1.05294e+01 2.00000e+01]
 [6.10000e-01 1.01616e+03 5.87650e+00 3.90000e+01]]


In [4]:
target = np.array([data['Apparent Temperature (C)'], data['Temperature (C)']],
                  dtype="float32").T
print(target)

[[ 7.388889  9.472222]
 [ 7.227778  9.355556]
 [ 9.377778  9.377778]
 ...
 [22.038889 22.038889]
 [21.522223 21.522223]
 [20.438889 20.438889]]


In [5]:
input = torch.from_numpy(input)
target = torch.from_numpy(target)
print(input)
print(target)

tensor([[8.9000e-01, 1.0151e+03, 1.4120e+01, 2.5100e+02],
        [8.6000e-01, 1.0156e+03, 1.4265e+01, 2.5900e+02],
        [8.9000e-01, 1.0159e+03, 3.9284e+00, 2.0400e+02],
        ...,
        [5.6000e-01, 1.0157e+03, 8.9838e+00, 3.0000e+01],
        [6.0000e-01, 1.0160e+03, 1.0529e+01, 2.0000e+01],
        [6.1000e-01, 1.0162e+03, 5.8765e+00, 3.9000e+01]])
tensor([[ 7.3889,  9.4722],
        [ 7.2278,  9.3556],
        [ 9.3778,  9.3778],
        ...,
        [22.0389, 22.0389],
        [21.5222, 21.5222],
        [20.4389, 20.4389]])


In [6]:
#declare random weight and bias
w = torch.randn(2, 4, requires_grad=True,dtype=torch.float32)
b = torch.randn(2, requires_grad=True, dtype=torch.float32)

In [7]:
# return (input*weight)+bias
def SModel(x):
    return x @ w.t() + b

In [8]:
#mean absolute error
def MAE(t1, t2):
    dif = torch.abs(t1 - t2)
    return torch.sum(dif)/dif.numel()

In [9]:
for i in range(30001):
    pred = SModel(input)
    loss = MAE(pred, target)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 0.00001
        b -= b.grad * 0.00001
        w.grad.zero_()
        b.grad.zero_()
    if (i%1000==0 or i==10000):
        print("Loss:{} at epoch:{}/30000".format(loss, i))

Loss:980.4078369140625 at epoch:0/30000
Loss:68.36116790771484 at epoch:1000/30000
Loss:47.74632263183594 at epoch:2000/30000
Loss:27.66729736328125 at epoch:3000/30000
Loss:12.721317291259766 at epoch:4000/30000
Loss:11.877906799316406 at epoch:5000/30000
Loss:11.812378883361816 at epoch:6000/30000
Loss:11.749520301818848 at epoch:7000/30000
Loss:11.68753433227539 at epoch:8000/30000
Loss:11.626368522644043 at epoch:9000/30000
Loss:11.565993309020996 at epoch:10000/30000
Loss:11.506383895874023 at epoch:11000/30000
Loss:11.447610855102539 at epoch:12000/30000
Loss:11.38960075378418 at epoch:13000/30000
Loss:11.332364082336426 at epoch:14000/30000
Loss:11.2758207321167 at epoch:15000/30000
Loss:11.22004222869873 at epoch:16000/30000
Loss:11.165005683898926 at epoch:17000/30000
Loss:11.110785484313965 at epoch:18000/30000
Loss:11.057377815246582 at epoch:19000/30000
Loss:11.004744529724121 at epoch:20000/30000
Loss:10.952868461608887 at epoch:21000/30000
Loss:10.901703834533691 at epoch

In [10]:
#Sample test input for random value
testInput = [[0.94, 1100, 15, 250]]
testInput = torch.tensor(testInput, dtype=torch.float32)
testPred = SModel(testInput)
print(testPred)

tensor([[17.0061, 16.3044]], grad_fn=<AddBackward0>)


# **Using Build-In Functions**

In [11]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
tstrain = TensorDataset(input, target)
dltrain = DataLoader(tstrain, batch_size=5, shuffle=True)

In [12]:
model = nn.Linear(4, 2)
model.bias

Parameter containing:
tensor([ 0.2158, -0.2763], requires_grad=True)

In [13]:
preds = model(input)
lossfn = nn.SmoothL1Loss()
loss = lossfn(preds, target)
op = torch.optim.SGD(model.parameters(), 0.00001)

In [14]:
def fit(epochs, model, lossfn, op, dltrain):
    for epoch in range(epochs):
        for i, j in dltrain:
            pred = model(i)
            loss = lossfn(pred, j)
            loss.backward()
            op.step()
            op.zero_grad()
        if (epoch%10==0 or epoch==epochs):
            print("Loss:{} at epoch:{}/{}".format(loss, epoch+1, epochs))

In [15]:
fit(30, model, lossfn, op, dltrain)

Loss:7.902092456817627 at epoch:1/30
Loss:9.365870475769043 at epoch:11/30
Loss:4.161231994628906 at epoch:21/30


In [16]:
model(torch.tensor([[0.91, 1661, 24, 222]]))

tensor([[ 9.6558, 12.6373]], grad_fn=<AddmmBackward>)